In [ ]:
%matplotlib inline

In [ ]:
import os, re
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

def set_plot_params(fontsize=12, font='NanumGothic'):
    home = os.path.expanduser('~')
    font_path = os.path.join(
        home, 'AppData/Local/Microsoft/Windows/Fonts'.replace('/', '\\'))
    font_files = fm.findSystemFonts(fontpaths=font_path)
    if hasattr(fm.fontManager, 'addfont'):
        for path in font_files:
            fm.fontManager.addfont(path=path)
    else:
        font_list = fm.creatFontList(font_files)
        fm.fontManager.ttflist.extend(font_list)
    plt.rcParams["font.family"] = font
    plt.rcParams["font.size"] = fontsize
    plt.rcParams["axes.unicode_minus"] = False

set_plot_params()

# 탐색

In [ ]:
data = pd.read_csv('data.csv')
data.to_pickle('data.p')

In [ ]:
# data['TBL'].unique()
# data['ITM_NM'].unique()
# data.groupby('ITM_NM')['UNIT_NM'].unique()
# data.groupby(['C1_NM', 'PRD_DE', 'TBL'])['ITM_NM'].count()



# x = data.pivot_table(index='C1_NM', columns='PRD_DE', values='DT',aggfunc=sum)
# x.to_clipboard()



# data.loc[data['C1_NM'].str.match('강력범죄')]

# loc = data['TBL'] == '서울지방검찰청'
# loc &= data['C1_NM'].str.match('강력범죄')
# loc &= data['PRD_DE'] == 2005

# data.loc[loc].sort_values(['C1_NM', 'ITM_NM']).reset_index(drop=True).to_clipboard()
# data.groupby(['PRD_DE', 'TBL'])['C2_NM'].apply(lambda x: len(x.unique()))
# data.groupby(['PRD_DE', 'TBL', 'C2_NM'])['C2_OBJ_NM'].apply(lambda x: len(x.unique())).to_clipboard()
# data.loc[data['PRD_DE'] == 2002]

# Refining: 구간별 데이터 형태 변환 통일

In [ ]:
data_pre = pd.read_pickle('data.p')
loc = data_pre['TBL'].eq('합계')
data_pre = data_pre.loc[~loc].copy()

In [ ]:
loc = data_pre['C1_NM'].str.match('총계')
data = data_pre.loc[~loc]

- `%`단위 데이터 빼기

In [ ]:
data_pct = data.loc[data['UNIT_NM'].eq('%')]
data1 = data.loc[~data['UNIT_NM'].eq('%')]

### C2 항목 필요할까? NO 그치만 뭔가 수상하다

In [ ]:
data['C2_NM'].unique()

In [ ]:
data['C2_OBJ_NM'].unique()

In [ ]:
loc = data['TBL'] == '서울지방검찰청'
loc &= data['PRD_DE'] == 2021
loc &= data['ITM_NM'] == '발생건수'
loc &= data['C1_NM'] == '형법범'

In [ ]:
data.loc[loc]

### 1992년 이전 데이터 C1항목 합치기

In [ ]:
data1_pre1992 = data1.loc[data1['PRD_DE'] <= 1992]

In [ ]:
data2_pre1992 = data1_pre1992.groupby(['TBL', 'ITM_NM', 'UNIT_NM', 'C1_NM', 'PRD_DE'])['DT'].sum().reset_index()

### 1993~2001년 데이터 C1항목 합치기

In [ ]:
loc = (data1['PRD_DE'] > 1992) & (data1['PRD_DE'] <= 2001)
data1_mid = data1.loc[loc].copy()

In [ ]:
data2_mid = data1_mid.groupby(['TBL', 'ITM_NM', 'UNIT_NM', 'C1_NM', 'PRD_DE'])['DT'].sum().reset_index()

### 2002년 이후 데이터 C1항목 합치기

In [ ]:
loc = data1['PRD_DE'] > 2001
data1_post2002 = data1.loc[loc].copy()

In [ ]:
data2_post2002 = data1_post2002.groupby(['TBL', 'ITM_NM', 'UNIT_NM', 'C1_NM', 'PRD_DE'])['DT'].sum().reset_index()

In [ ]:
# data2_post2002.loc[
#     data2_post2002['TBL'].eq('합계') & \
#     data2_post2002['C1_NM'].eq('횡령') & \
#     data2_post2002['ITM_NM'].eq('검거건수')].set_index('PRD_DE')['DT']

In [ ]:
data2 = pd.concat([data2_pre1992, data2_mid, data2_post2002], axis=0).reset_index(drop=True)
# data2.to_csv('건수로만_C2항목제거_data.csv', index=False)
data2.to_pickle('data2.p')

In [ ]:
# pd.read_csv('건수로만_C2항목제거_data.csv')

In [ ]:
# data2_pivot = data2.pivot_table(index='C1_NM', columns='PRD_DE', values='DT', aggfunc=sum)

In [ ]:
# data2_pivot.to_clipboard()

# Refining 2 : 중복 명칭 합치기

In [ ]:
data2 = pd.read_pickle('data2.p')

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('.+음란')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '기타 음란행위'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('.+노동관계')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '노동조합 및 노동관계 조정법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('농수산물')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '농수산물 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('.?담배')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '담배관련'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('마약류관리에 관한.+\(대마\)')
loc |= data2['C1_NM'].str.match('.?대마')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '마약류_대마'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('마약류관리에 관한.+\(마약\)')
# loc |= data2['C1_NM'].str.match('.?대마')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '마약류_마약'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('마약류관리에 관한.+\(향정')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '마약류_향정신'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('마약류관리에.+법률$')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '마약류_unknown'

In [ ]:
loc = data2['C1_NM'].str.match('마약법')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '마약류_unknown'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('물환경보전법')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '물환경보전법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('방화치')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '방화'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('배임수재|배임증재')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '배임'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('병역법')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '병역법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('부동산 소유권 이전')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '부동산 소유권 이전 등기등에 관한 법률'

In [ ]:
loc = data2['C1_NM'].str.match('부동산실권리자 명의등기에 관한법률')
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '부동산 소유권 이전 등기등에 관한 법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('부정경쟁방지')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '부정경쟁방지법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('산림자원의')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '산림자원의 조성 및 관리에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('살인')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '살인'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('상해')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '상해'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('석유')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '석유관련법석유관련법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('성매매')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '성매매'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('수산업법|수산자원')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '수산업법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('스토킹')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '스토킹 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('신용.+경매')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '신용업무경매'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('신용카드|신용정보')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '신용카드업법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('아동')
loc &= ~data2['C1_NM'].str.match('아동복지')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '아동청소년의 성보호에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('약취')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '약취와 유인'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('약취')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '약취와 유인'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('업무상')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '업무상과실치사'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('오수')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '오수 분뇨 및 축산폐수의 처리에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('옥외광고물')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '옥외광고물등관리법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('외국환')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '외국환거래법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('위증')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '위증과증거인멸'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('유가증권')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '유가증권 인지 우표'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('유기|유기치')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '유기'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('유해')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '유해화학물질관리법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('윤락행위')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '윤락행위등방지법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('음반')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '음반 비디오물 및 게임물에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('음용')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '음용수에관한죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('일수와')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '일수와수리에관한죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('자동차')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '자동차관리법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('정보통신망')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '정보통신망이용촉진 등에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('조세범')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '조세범처벌법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('조수보호')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '조수보호 및 수렵에관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('주거\s?침')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '주거침입'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('^청소년')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '청소년보호법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('체포와')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '체포와감금'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('체포치')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '체포치사상'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('총계')

In [ ]:
data2 = data2.loc[~loc]

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('총포')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '총포 도검 화약류 단속법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('촬영물')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '촬영물등 이용협박 강요'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('축산물')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '축산물위생처리법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('특가법')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '특가법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('특별법')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '특별법범'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('폭력행위')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '폭력행위 등'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('폭행')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '폭행'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('풍속')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '풍속범죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('학원의')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '학원의설립운영 및 과외교습에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('허위영상')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '허위영상물 편집 반포등'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('화재')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '화재예방 소방시설설치유지 및 안전관리에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('강도')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '강도 관련'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('강간')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '강간 관련'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('가정')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '가정폭력범죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('개발제한')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '개발제한구역 관련'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('건설기계')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '건설기계관리법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('공무')
loc &= ~data2['C1_NM'].str.match('공무원')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '공무방해'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('공무')
loc &= ~data2['C1_NM'].str.match('공무원')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '공무방해'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('공직')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '공직선거법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('과실치')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '과실치사상'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('교통사고')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '교통사고 처리특례법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('권리행사')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '권리행사 방해'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('감금치')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '감금치상사'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('강력범')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '강력범죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('건설')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '건설산업기본법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('공안')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '공안을 해하는죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('공중위생')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '공중위생법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('과실')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '과실범죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('내란의')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '내란의 죄'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('대부업')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '대부업등의 등록 및 금융이용자보호에 관한법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('도로교')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '도로교통법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('도박')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '도박'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('도시')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '도시및주거환경정비법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('도주')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '도주와 범인 은닉'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('부동산.+법률')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '부동산 소유권 이전 등기등에 관한 법률'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('보건')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '보건범죄 특조법'

--------------------------------------------

In [ ]:
loc = data2['C1_NM'].str.match('손괴')

In [ ]:
data2.loc[loc, 'C1_NM'].unique()

In [ ]:
data2.loc[loc, 'C1_NM'] = '손괴'

### Merge Data Values

In [ ]:
data3 = data2.groupby(['TBL', 'ITM_NM', 'UNIT_NM', 'C1_NM', 'PRD_DE'])['DT'].sum().reset_index()

In [ ]:
data3.to_pickle('data3.p')

# 일관되지 못한 ITM_NM 찾고 제거하기

In [ ]:
data3['TBL'].unique()

In [ ]:
data3_pre = data3.loc[data3['PRD_DE'] < 2002]
data3_mid = data3.loc[(data3['PRD_DE'] >= 2002) & (data3['PRD_DE'] < 2014)]
data3_post = data3.loc[data3['PRD_DE'] >= 2014]

In [ ]:
data3_pre.groupby('TBL')['ITM_NM'].apply(lambda x: len(x.unique()))

In [ ]:
data3_mid.groupby('TBL')['ITM_NM'].apply(lambda x: len(x.unique()))

In [ ]:
data3_post.groupby('TBL')['ITM_NM'].apply(lambda x: len(x.unique()))

In [ ]:
data3_pre['ITM_NM'].unique()

In [ ]:
data3_mid['ITM_NM'].unique()

In [ ]:
data3_post['ITM_NM'].unique()

> 2014년 이후 부터는 `미상검거인원`, `법인` 항목이 추가된다 -> 필요 없으므로 제거

In [ ]:
data4 = data3[~data3['ITM_NM'].isin(['미상검거인원', '법인'])].copy()

In [ ]:
# data4.to_csv('건수_범죄명통일.csv', index=False)
# data4.to_excel('건수_범죄명통일_utf.xlsx', index=False)

In [ ]:
data4.to_pickle('data4.p')

# 인구 추가

In [ ]:
rgn_map = pd.read_csv('item_rgn_map.csv', index_col='item')['rgn']
tbl_rgn_map = {x:rgn_map[x[:-5]] for x in data4['TBL'].unique()}

In [ ]:
data5 = data4.copy()
data5['RGN'] = data5['TBL'].map(tbl_rgn_map)

In [ ]:
data5

In [ ]:
data5 = data5.groupby(['RGN', 'ITM_NM', 'UNIT_NM', 'C1_NM', 'PRD_DE'])['DT'].sum().reset_index()

In [ ]:
pop = pd.read_csv('pop.csv', index_col=['Year', 'Type'], header=0)

In [ ]:
pop_tot = pop.rename_axis(['PRD_DE', 'Type'], axis=0) \
    .swaplevel().sort_index().loc['인구'].unstack()
pop_m = pop.rename_axis(['PRD_DE', 'Type'], axis=0) \
    .swaplevel().sort_index().loc['남'].unstack()
pop_f = pop.rename_axis(['PRD_DE', 'Type'], axis=0) \
    .swaplevel().sort_index().loc['여'].unstack()
pop_wide = pd.concat([pop_m.rename('남'), pop_f.rename('여'), pop_tot.rename('인구')], axis=1) \
    .rename_axis(['RGN', 'PRD_DE'])

data5 = data5.merge(pop_wide.reset_index(), on=['PRD_DE', 'RGN'], how='left')

In [ ]:
prd_de_rm = data5.loc[data5['인구'].isnull(), 'PRD_DE'].unique()
prd_de_rm.sort()

data5 = data5.loc[data5['인구'].notnull()]

In [ ]:
data5['ITM_NM'].unique()

In [ ]:
data5_arrest = data5.loc[data5['ITM_NM'].eq('검거건수')]
# data5_arrest.groupby(['C1_NM', 'RGN'])['DT'].apply(lambda x: 1 if x.mean() > 0 else 0).unstack().to_clipboard()
# data5_arrest.groupby(['C1_NM', 'RGN'])['DT'].apply(lambda x: x.mean()).unstack().to_clipboard()
data5_arrest_2014 = data5_arrest.loc[data5_arrest['PRD_DE'] >= 2014]

In [ ]:
data6 = data5_arrest_2014.loc[~data5_arrest_2014['C1_NM'].isin(['특별법범', '형법범'])]

In [ ]:
data6.to_pickle('data6.p')

In [ ]:
data6.to_csv('data6.csv', index=False)